In [1]:
import spacy
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [2]:
doc = nlp("A leak was noticed from the RCP pump 1A.")
spacy.displacy.render(doc, style="dep")

In [3]:
doc = nlp("The shaft vibration appears to be causing the motor to vibrate")
spacy.displacy.render(doc, style="dep")

In [66]:
doc[1].doc

A leak was noticed from the RCP pump 1A.

In [59]:
doc = nlp("RCP pump 1A pressure gauge was not responding ,")
spacy.displacy.render(doc, style="dep")
print(doc[0:9].root.nbor())

,


In [4]:
doc = nlp("RCP pump 1A pressure gauge was found not operating.")
spacy.displacy.render(doc, style="dep")
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head, [i for i in token.lefts], [i for i in token.rights], [i for i in token.children])

RCP NOUN compound gauge [] [] []
pump NOUN compound gauge [] [] []
1A NOUN compound gauge [] [] []
pressure NOUN compound gauge [] [] []
gauge NOUN nsubjpass found [RCP, pump, 1A, pressure] [] [RCP, pump, 1A, pressure]
was AUX auxpass found [] [] []
found VERB ROOT found [gauge, was] [operating, .] [gauge, was, operating, .]
not PART neg operating [] [] []
operating VERB xcomp found [not] [] [not]
. PUNCT punct found [] [] []


In [67]:
doc = nlp("RCP pump 1A pressure gauge was found inoperative.")
spacy.displacy.render(doc, style="dep")

In [44]:
doc[1:3].doc

Pump power supply has been found burnout.

In [6]:
doc = nlp("Rupture of pump bearings caused shaft degradation.")
spacy.displacy.render(doc, style="dep")
for token in doc:
    print(token.text, token.dep_, token.head, token.head.pos_, token.head.head)

Rupture nsubj caused VERB caused
of prep Rupture NOUN caused
pump compound bearings NOUN of
bearings pobj of ADP Rupture
caused ROOT caused VERB caused
shaft compound degradation NOUN caused
degradation dobj caused VERB caused
. punct caused VERB caused


In [7]:
doc = nlp("Rupture of pump bearings caused shaft degradation and consequent flow reduction.")
spacy.displacy.render(doc, style="dep")

In [24]:
doc = nlp("Pump power supply has been found burnout.")
spacy.displacy.render(doc, style="dep")

In [28]:
doc[1:3].head

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'head'

In [19]:
doc = nlp("Pump test failed due to power supply failure.")
spacy.displacy.render(doc, style="dep")

In [23]:
list(doc[-2].lefts)


[supply]

In [10]:
doc = nlp("Pump inspection revealed excessive impeller degradation.")
spacy.displacy.render(doc, style="dep")

In [11]:
doc = nlp("Pump inspection revealed excessive impeller degradation likely due to cavitation.")
spacy.displacy.render(doc, style="dep")

In [15]:
doc = nlp("Pump inspection revealed excessive impeller degradation likely due to cavitation.")
spacy.displacy.render(doc, style="ent")

/Users/wangc/miniconda3/envs/sr2ml_libs/lib/python3.9/site-packages/spacy/displacy/__init__.py:192: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [13]:
if nlp.has_pipe('entity_ruler'):
    nlp.remove_pipe('entity_ruler')
if nlp.has_pipe('merge_entities'):
    nlp.remove_pipe('merge_entities')

ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MyCorp Inc."}]
ruler.add_patterns(patterns)
nlp.add_pipe("merge_entities")

doc = nlp("MyCorp Inc. is a company in the U.S.")
print([(ent.text, ent.label_) for ent in doc.ents])
spacy.displacy.render(doc, style='ent')

[('MyCorp Inc.', 'ORG'), ('U.S.', 'GPE')]


In [14]:
for ent in doc.ents:
    print(ent.label_)

ORG
GPE


In [14]:

spacy.displacy.render(doc, style='dep')

In [15]:
doc = nlp("Alex Smith worked at Acme Corp Inc.")
# If you're not in a Jupyter / IPython environment, use displacy.serve
spacy.displacy.render(doc, options={"fine_grained": True})

## Example that can be used for Relation Extraction after Named Entity Recognizer 

In [16]:
import spacy
from spacy.language import Language
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

# @Language.component("extract_person_orgs")
# def extract_person_orgs(doc):
#     person_entities = [ent for ent in doc.ents if ent.label_ == "PERSON"]
#     for ent in person_entities:
#         head = ent.root.head
#         if head.lemma_ == "work":
#             preps = [token for token in head.children if token.dep_ == "prep"]
#             for prep in preps:
#                 orgs = [token for token in prep.children if token.ent_type_ == "ORG"]
#                 print({'person': ent, 'orgs': orgs, 'past': head.tag_ == "VBD"})
#     return doc

@Language.component("extract_person_orgs")
def extract_person_orgs(doc):
    person_entities = [ent for ent in doc.ents if ent.label_ == "PERSON"]
    for ent in person_entities:
        head = ent.root.head
        if head.lemma_ == "work":
            preps = [token for token in head.children if token.dep_ == "prep"]
            for prep in preps:
                orgs = [t for t in prep.children if t.ent_type_ == "ORG"]
                aux = [token for token in head.children if token.dep_ == "aux"]                
                past_aux = any(t.tag_ == "VBD" for t in aux)                
                past = head.tag_ == "VBD" or head.tag_ == "VBG" and past_aux                
                print({'person': ent, 'orgs': orgs, 'past': past})
    return doc

# To make the entities easier to work with, we'll merge them into single tokens
nlp.add_pipe("merge_entities")
nlp.add_pipe("extract_person_orgs")

doc = nlp("Alex Smith worked at Acme Corp Inc.")
# If you're not in a Jupyter / IPython environment, use displacy.serve
displacy.render(doc, options={"fine_grained": True})

{'person': Alex Smith, 'orgs': [Acme Corp Inc.], 'past': True}


## Coreferee Resolution: require spacy>=3.1.0,<3.2.0
WIP PR for spacy to integrate both coreferee and nueralcoef into spacy
https://github.com/explosion/spaCy/pull/7264

In [17]:
import coreferee, spacy
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('coreferee')

In [18]:
doc = nlp("Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much.")

/Users/wangc/miniconda3/envs/sr2ml_libs/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [19]:
doc._.coref_chains.print()

0: he(1), his(6), Peter(9), He(16), his(18)
1: work(7), it(14)
2: [He(16); wife(19)], they(21), They(26), they(31)
3: Spain(29), country(34)


In [22]:
for token in doc:
    if token._.coref_chains:
        # print(token._.coref_chains)
        for chain in token._.coref_chains:
            for tk in chain:
                print(doc[tk[0]])
        ccc
            

he
his
Peter
He
his


NameError: name 'ccc' is not defined

In [23]:
for ent in doc.ents:
    print(ent.root.head)

had
to


In [24]:
doc[0]

Although

In [25]:
doc[16]._.coref_chains.print()

0: he(1), his(6), Peter(9), He(16), his(18)
2: [He(16); wife(19)], they(21), They(26), they(31)


In [26]:
doc._.coref_chains.resolve(doc[31])

[Peter, wife]

#### chains and mentions can be navigated much as if they were lists 

In [27]:
for chain in doc._.coref_chains:
    for mention in chain:
        print(mention)

[1]
[6]
[9]
[16]
[18]
[7]
[14]
[16, 19]
[21]
[26]
[31]
[29]
[34]


In [28]:
doc._.coref_chains[1].pretty_representation

'1: work(7), it(14)'

## Example from https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch12/Knowledge_Graph.ipynb

In [29]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.tokens import Span

nlp = spacy.load('en_core_web_sm')
print(*nlp.pipeline, sep='\n')

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x14de7fdb0>)
('tagger', <spacy.pipeline.tagger.Tagger object at 0x14e052a40>)
('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x13b55bb80>)
('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x14e088200>)
('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x14e0a8d00>)
('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x145970100>)


In [30]:
from spacy.tokens import Span
from spacy.language import Language

@Language.component("normEntities")
def norm_entities(doc):
    ents = []
    for ent in doc.ents:
        if ent[0].pos_ == "DET": # leading article
            ent = Span(doc, ent.start+1, ent.end, label=ent.label)
        if len(ent) > 0:
            if ent[-1].pos_ == "PART": # trailing particle like 's
                ent = Span(doc, ent.start, ent.end-1, label=ent.label)
            if len(ent) > 0:
                ents.append(ent)
    doc.ents = tuple(ents)
    return doc

In [31]:
nlp = spacy.load("en_core_web_lg")
# nlp.add_pipe("normEntities")

In [32]:
doc = nlp("Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much. Peter's wife is pretty")

In [33]:
doc1=nlp(doc.text)

In [34]:
doc1

Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much. Peter's wife is pretty

In [35]:
for ent in doc1.ents:
    print(ent.label_)

PERSON
GPE
PERSON


In [36]:
pipes = [pipe for (pipe,_) in nlp.pipeline]

In [37]:
pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [38]:
for pipe in pipes:
    _ = nlp.remove_pipe(pipe)

In [39]:
nlp.add_pipe("normEntities")

<function __main__.norm_entities(doc)>

In [40]:
doc2 = nlp(doc1)

TypeError: Argument 'string' has incorrect type (expected str, got spacy.tokens.doc.Doc)

In [41]:
doc2 = norm_entities(doc1)

In [42]:
doc2

Although he was very busy with his work, Peter had had enough of it. He and his wife decided they needed a holiday. They travelled to Spain because they loved the country very much. Peter's wife is pretty

In [43]:
doc = nlp("Pump made noises - not enough to affect performance.")
spacy.displacy.render(doc, style="dep")
for tk in doc:
    print(list(tk.subtree))

/Users/wangc/miniconda3/envs/sr2ml_libs/lib/python3.9/site-packages/spacy/displacy/__init__.py:128: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


[Pump]
[made]
[noises]
[-]
[not]
[enough]
[to]
[affect]
[performance]
[.]


In [44]:
doc = nlp("The pump shaft vibration appears to be causing the motor to vibrate.")
spacy.displacy.render(doc, style="dep", options={'compact':False})